### Contextualizing Transcriptomic Data

In [1]:
from riptide import *

### Testing with Toy Model

In [2]:
# Load in example model
toy_model = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/reconstructions/example_GENRE.sbml')

# gene1 = Glucose transporter
# gene2 = Proline transporter
# gene3 = Glycine transporter
# gene4 = Hydrogen efflux
# gene5 = Carbon dioxide efflux
# gene6 = Phosphate transporter
# gene7 = Glycolysis
# gene8 = Stickland fermentation

In [4]:
toy_model

Name,iTEST
Memory address,0x07f9715d9fe80
Number of metabolites,14
Number of reactions,16
Number of groups,0
Objective expression,1.0*DM_atp_c - 1.0*DM_atp_c_reverse_1b037
Compartments,"e, c"


In [18]:
for cpd in toy_model.metabolites:
    print(cpd.id + '\t' + cpd.name)

gluc_e	Glucose
gluc_c	Glucose
pro_e	Proline
pro_c	Proline
gly_e	Glycine
gly_c	Glycine
h_e	H+
h_c	H+
co2_e	CO2
co2_c	CO2
pi_e	Pi
pi_c	Pi
adp_c	ADP
atp_c	ATP


In [6]:
# Find most parsimonious route of flux
from cobra.flux_analysis.parsimonious import pfba
toy_solution = pfba(toy_model)
print(toy_solution.fluxes)

EX_gluc_e   -1000.0
rxn1         1000.0
EX_pro_e        0.0
rxn2            0.0
EX_gly_e        0.0
rxn3            0.0
EX_h_e       1000.0
rxn4         1000.0
EX_co2_e        0.0
rxn5            0.0
EX_pi_e     -1000.0
rxn6         1000.0
EX_adp_c    -1000.0
rxn7         1000.0
rxn8            0.0
DM_atp_c     1000.0
Name: fluxes, dtype: float64


In [3]:
# Create associated transcriptomes
glucose_transcriptome = {'gene1':100, 'gene2':1, 'gene3':1, 'gene4':1, 
                         'gene5':1, 'gene6':100, 'gene7':10000, 'gene8':1}
peptide_transcriptome = {'gene1':1, 'gene2':100, 'gene3':100, 'gene4':1, 
                         'gene5':1, 'gene6':1, 'gene7':1, 'gene8':10000}

In [4]:
# Contextualize toy model
toy_model_glucose = riptide.contextualize(toy_model, transcriptome=glucose_transcriptome)


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 9 from 16 (43.75% change)
Metabolites pruned to 8 from 14 (42.86% change)

RIPTiDe completed in 5 seconds



In [5]:
for cpd in toy_model_glucose.model.metabolites:
    print(cpd.id + '\t' + cpd.name)

gluc_e	Glucose
gluc_c	Glucose
h_e	H+
h_c	H+
pi_e	Pi
pi_c	Pi
adp_c	ADP
atp_c	ATP


In [6]:
# Contextualize toy model
toy_model_peptide = riptide.contextualize(toy_model, transcriptome=peptide_transcriptome)


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 13 from 16 (18.75% change)
Metabolites pruned to 12 from 14 (14.29% change)

RIPTiDe completed in 5 seconds



In [7]:
for cpd in toy_model_peptide.model.metabolites:
    print(cpd.id + '\t' + cpd.name)

pro_e	Proline
pro_c	Proline
gly_e	Glycine
gly_c	Glycine
h_e	H+
h_c	H+
co2_e	CO2
co2_c	CO2
pi_e	Pi
pi_c	Pi
adp_c	ADP
atp_c	ATP


### Testing with E.coli K-12 MG1655 model

In [2]:
iJO1366 = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/reconstructions/iJO1366.xml')
iJO1366.objective = iJO1366.reactions.BIOMASS_Ec_iJO1366_WT_53p95M

# Open all exchanges
exchanges = set()
for rxn in iJO1366.reactions:
    if len(rxn.reactants) == 0 or len(rxn.products) == 0:
        rxn.bounds = (min(rxn.lower_bound, -1000.), max(rxn.upper_bound, 1000.))
        exchanges |= set([rxn.id])

In [18]:
iJO1366

Name,iJO1366
Memory address,0x07f8f897aa3d0
Number of metabolites,1805
Number of reactions,2583
Objective expression,0.0 + 1.0*BIOMASS_Ec_iJO1366_WT_53p95M - 1.0*BIOMASS_Ec_iJO1366_WT_53p95M_reverse_06c4a
Compartments,"periplasm, cytosol, extracellular space"


In [3]:
def max_doubling_time(model):
    
    with model as m: 
        growth =  m.slim_optimize()
    
    if growth < 1e-6:
        growth = 'No growth'
    else:    
        growth = (1.0 / growth) * 3600.0
        if growth < 60.0:
            growth = str(round(growth, 1)) + ' minutes'
        else:
            growth = growth / 60.0
            growth = str(round(growth, 3)) + ' hours'
            
    print(growth)


def collect_doubling_times(flux_samples, biomass):
    biomass = list(flux_samples[biomass])
    times = []
    
    for x in biomass:
        growth = (1.0 / x) * 3600.0 # Calculated in minutes
        growth = round(growth, 2)
        times.append(growth)
        
    return times


def collect_growth_rates(flux_samples, biomass):
    biomass = list(flux_samples[biomass])
    rates = []
    
    for x in biomass:
        rate = x / 60.0
        rate = round(rate, 3)
        rates.append(rate)
        
    return rates

In [7]:
max_doubling_time(iJO1366)

34.0 minutes


### Flux sampling on base model

In [8]:
# Prune blocked reactions
flux_span = flux_variability_analysis(iJO1366, fraction_of_optimum=0.01)
blocked_rxns = []
for rxn_id, min_max in flux_span.iterrows():
    if max(abs(min_max)) < 1e-6: blocked_rxns.append(rxn_id)
for rxn in blocked_rxns: 
    iJO1366.reactions.get_by_id(rxn).remove_from_model(remove_orphans=True)

In [11]:
# Constrain possible solutions
obj_val = iJO1366.slim_optimize()
obj_constraint = iJO1366.problem.Constraint(iJO1366.objective.expression, lb=obj_val*0.8, ub=obj_val)
iJO1366.add_cons_vars([obj_constraint])
iJO1366.solver.update()

In [13]:
# Flux sampling of base model
iJO1366_sampling_object = ACHRSampler(iJO1366)
iJO1366_flux_samples = iJO1366_sampling_object.sample(5000)

[-69727.61, 92.4, 665273.68]


In [40]:
# Collect base model growth information
base_rates = collect_growth_rates(iJO1366_flux_samples, 'BIOMASS_Ec_iJO1366_WT_53p95M')

# Screen data against negative values
screened_base_rates = []
for x in base_rates:
    if x > 0.0: 
        screened_base_rates.append(str(x))

In [42]:
# Save times to a file
with open('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/unconstrained_growth_rates.tsv', 'w') as rates:
    for x in screened_base_rates: rates.write(x + '\n')

### Contextualizing Published Datasets

In [3]:
# Read in transcriptomes

# Data collected from:
# Monk et al. (2016). Multi-omics Quantification of Species Variation of Escherichia coli
# Links Molecular Features with Strain Phenotypes. Cell Systems. 3; 238–251.

# Load in GPR translations
gpr_dict = {}
with open('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/transcript/Monk_et_al_2016/iJO1366_genes.tsv', 'r') as genes:
    for line in genes:
        line = line.split()
        gpr_dict[line[1]] = line[0]

# Normalized abundances
# Separate into treatment goups and calculate medians
m9_aerobic = {}
m9_anaerobic = {}     
with open('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/transcript/Monk_et_al_2016/normalized.tsv', 'r') as transcription:
    for line in transcription:
        line = line.split()
        if line[0] == 'gene':
            continue
        else:
            try:
                gene = gpr_dict[line[0]]
            except:
                continue
            m9_aerobic[gene] = numpy.median([int(x) for x in line[1:4]])
            m9_anaerobic[gene] = numpy.median([int(y) for y in line[4:7]])

# Rich media (LB) data from:
# Double-stranded transcriptome of E. coli
# Meghan Lybecker, Bob Zimmermann, Ivana Bilusic, Nadezda Tukhtubaeva, Renée Schroeder
# Proceedings of the National Academy of Sciences Feb 2014, 111 (8) 3134-3139; DOI: 10.1073/pnas.1315974111
lb_aerobic = {}     
with open('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/transcript/Lybecker_2014.mapped.norm.tsv', 'r') as transcription:
    header = transcription.readline()
    for line in transcription: 
        line = line.split()
        lb_aerobic[line[0]] = float(line[1])

### Sensitivity testing for minimum objective flux

In [7]:
iJO1366_10 = riptide.contextualize(iJO1366, transcriptome=m9_aerobic, fraction = 0.1)
iJO1366_20 = riptide.contextualize(iJO1366, transcriptome=m9_aerobic, fraction = 0.2)
iJO1366_30 = riptide.contextualize(iJO1366, transcriptome=m9_aerobic, fraction = 0.3)
iJO1366_40 = riptide.contextualize(iJO1366, transcriptome=m9_aerobic, fraction = 0.4)
iJO1366_50 = riptide.contextualize(iJO1366, transcriptome=m9_aerobic, fraction = 0.5)
iJO1366_60 = riptide.contextualize(iJO1366, transcriptome=m9_aerobic, fraction = 0.6)
iJO1366_70 = riptide.contextualize(iJO1366, transcriptome=m9_aerobic, fraction = 0.7)
iJO1366_80 = riptide.contextualize(iJO1366, transcriptome=m9_aerobic, fraction = 0.8)
iJO1366_90 = riptide.contextualize(iJO1366, transcriptome=m9_aerobic, fraction = 0.9)


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 445 from 2583 (82.77% change)
Metabolites pruned to 451 from 1805 (75.01% change)
Flux through the objective DECREASED to ~20.38 from ~105.77 (80.73% change)
Contextualized metabolism has a concordancy of 39.0% (p<0.001) with the transcriptome

RIPTiDe completed in 37 seconds


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 446 from 2583 (82.73% change)
Metabolites pruned to 451 from 1805 (75.01% change)
Flux through the objective DECREASED to ~21.99 from ~105.77 (79.21% change)
Contextualized metabolism has a concordancy of 38.6% (p<0.001) with the transcriptome

RIPTiDe completed in 36 seconds


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific f

ValueError: Cannot take a larger sample than population when 'replace=False'

### Comparing to exchanged-constrained pFBA

In [4]:
# Set M9 media components + cofactors
iJO1366_m9 = copy.deepcopy(iJO1366)

#minimal_medium(iJO1366_m9, 0.1, minimize_components=True)

m9_components = ['EX_o2_e', 
                 'EX_glc__D_e', 
                 'EX_mg2_e', 
                 'EX_so4_e', 
                 'EX_ca2_e', 
                 'EX_cl_e', 
                 'EX_ca2_e', 
                 'EX_h2o_e', 
                 'EX_na1_e', 
                 'EX_k_e', 
                 'EX_h_e', 
                 'EX_pi_e', 
                 'EX_nh4_e',
                 
                 # Trace co-factors (essential)
                 'EX_adocbl_e',
                 'EX_cobalt2_e',
                 'EX_cu2_e',
                 'EX_fe3dcit_e',
                 'EX_mn2_e',
                 'EX_mobd_e',
                 'EX_ni2_e',
                 'EX_zn2_e']

# Close non-defined exchanges and collect reactants
m9_substrates = []
for rxn in exchanges:
    if not rxn in m9_components:
        iJO1366_m9.reactions.get_by_id(rxn).bounds = (0., max(iJO1366_m9.reactions.get_by_id(rxn).upper_bound, 1000.))
    else:
        m9_substrates.append(iJO1366_m9.reactions.get_by_id(rxn).reactants[0].id)
m9_substrates = set(m9_substrates)

In [5]:
# Constrain objective flux

# Base model
iJO1366_max = copy.deepcopy(iJO1366)
prev_obj_val = iJO1366_max.slim_optimize()
prev_obj_constraint = iJO1366_max.problem.Constraint(iJO1366_max.objective.expression, lb=prev_obj_val*0.8, ub=prev_obj_val)
iJO1366_max.add_cons_vars([prev_obj_constraint])
iJO1366_max.solver.update()

# M9 constraints
iJO1366_m9_max = copy.deepcopy(iJO1366_m9)
prev_obj_val = iJO1366_m9_max.slim_optimize()
prev_obj_constraint = iJO1366_m9_max.problem.Constraint(iJO1366_m9_max.objective.expression, lb=prev_obj_val*0.8, ub=prev_obj_val)
iJO1366_m9_max.add_cons_vars([prev_obj_constraint])
iJO1366_m9_max.solver.update()

In [7]:
# gapsplit flux sampler
# Keaty TC & Jensen PA (2019). gapsplit: Efficient random sampling for non-convex constraint-based models.
# bioRxiv 652917; doi: https://doi.org/10.1101/652917 

def gapsplit(model, n=500):

    # Define a few more variables
    max_tries=1000
    primary_tol=0.001
    secondary_frac=0.05
    min_range=1e-5
    enforce_range=True

    reactions = model.reactions
    fva = flux_variability_analysis(model, reactions, fraction_of_optimum=0.001)

    if secondary_frac >= 1.0:
        n_secondary = secondary_frac
    else:
        n_secondary = numpy.floor(secondary_frac * len(model.reactions)).astype(int)

    idxs = (fva.maximum - fva.minimum >= min_range).to_numpy().nonzero()[0]
    weights = (1.0 / (fva.maximum - fva.minimum) ** 2).to_numpy()

    samples = numpy.zeros((n, len(model.reactions)))
    k = 0
    infeasible_count = 0

    for try_ in range(max_tries):
        relative, target, width = maxgap(samples[0:k,idxs], fva.iloc[idxs,:])
        
        primary_var = numpy.argmax(relative)
        primary_target = target[primary_var]
        primary_lb = primary_target - primary_tol*width[primary_var]
        primary_ub = primary_target + primary_tol*width[primary_var]

        secondary_vars = numpy.random.choice(len(idxs), n_secondary, replace=False)
        secondary_targets = target[secondary_vars]
        secondary_weights = weights[idxs[secondary_vars]]

        new_sample = generate_sample(
            model, idxs[primary_var], primary_lb, primary_ub,
            idxs[secondary_vars], secondary_targets, secondary_weights)
        if new_sample is not None:
            if enforce_range:
                new_sample[new_sample > fva.maximum] = fva.maximum[new_sample > fva.maximum]
                new_sample[new_sample < fva.minimum] = fva.minimum[new_sample < fva.minimum]

            samples[k,:] = new_sample
            k += 1
        else:
            infeasible_count += 1

        if k >= n: break

    if k < n:
        samples = samples[:k,:]
    return pandas.DataFrame(data=samples,columns=fva.maximum.index)


def generate_sample(
        model, primary_var, primary_lb, primary_ub,
        secondary_vars=None, secondary_targets=None, secondary_weights=None):
    with model:
        model.reactions[primary_var].lower_bound = primary_lb
        model.reactions[primary_var].upper_bound = primary_ub
        model.objective = symengine.RealDouble(0)
        solution = model.optimize()
        if solution.status != 'optimal':
            return None
        else:
            return solution.fluxes


def maxgap(points, fva):
    points = points.copy()
    points = numpy.vstack((fva.minimum, points, fva.maximum))
    points.sort(0)

    gaps = points[1:,:] - points[0:-1,:]
    width = gaps.max(0)
    loc = gaps.argmax(0)
    left = numpy.zeros(width.size)
    for i in range(width.size):
        left[i] = points[loc[i],i]

    relative = width / (points[-1,:] - points[0,:])
    target = left + width / 2.0
    
    
    return relative, target, width


In [8]:
# Flux sample unconstrained base model
raw_flux_samples = gapsplit(iJO1366)

In [9]:
# Flux sample M9-constrained base model
raw_m9_flux_samples = gapsplit(iJO1366_m9)

In [10]:
# Flux sample objective flux constrained, M9-constrained model
max_m9_flux_samples = gapsplit(iJO1366_m9_max)

In [11]:
# Flux sample objective flux constrained
max_flux_samples = gapsplit(iJO1366_max)

In [12]:
# Get RIPTiDe flux samples without exchange constraints
iJO1366_m9_riptide = riptide.contextualize(iJO1366, transcriptome=m9_aerobic)
riptide_m9_flux_samples = iJO1366_m9_riptide.flux_samples


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 485 from 2583 (81.22% change)
Metabolites pruned to 481 from 1805 (73.35% change)
Flux through the objective DECREASED to ~85.9 from ~105.77 (18.79% change)
Context-specific metabolism correlates with transcriptome (p<<0.001 ***)

RIPTiDe completed in 1 minutes and 33 seconds



In [13]:
# Get RIPTiDe flux samples with M9 constraints
iJO1366_m9_riptide_media = riptide.contextualize(iJO1366_m9, transcriptome=m9_aerobic)
riptide_m9_media_flux_samples = iJO1366_m9_riptide_media.flux_samples


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 557 from 2583 (78.44% change)
Metabolites pruned to 561 from 1805 (68.92% change)
Flux through the objective DECREASED to ~31.22 from ~39.66 (21.28% change)
Context-specific metabolism correlates with transcriptome (p<<0.001 ***)

RIPTiDe completed in 1 minutes and 15 seconds



In [16]:
# Subset to just exchange levels and write to files
exchanges_x = set([x.id for x in iJO1366_m9_riptide_media.model.reactions if 'EX_' in x.id])
exchanges_y = set([y.id for y in iJO1366_m9_riptide.model.reactions if 'EX_' in y.id])
exchanges = exchanges_x.intersection(exchanges_y)

raw_flux_samples = raw_flux_samples.loc[0:, exchanges]
raw_m9_flux_samples = raw_m9_flux_samples.loc[0:, exchanges]
max_m9_flux_samples = max_m9_flux_samples.loc[0:, exchanges]
max_flux_samples = max_flux_samples.loc[0:, exchanges]
riptide_m9_media_flux_samples = riptide_m9_media_flux_samples.loc[0:, exchanges]
riptide_m9_flux_samples = riptide_m9_flux_samples.loc[0:, exchanges]

raw_flux_samples.to_csv('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/exchanges/raw.iJO1366.exchange_fluxes.tsv', sep='\t')
raw_m9_flux_samples.to_csv('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/exchanges/raw_m9.iJO1366.exchange_fluxes.tsv', sep='\t')
max_m9_flux_samples.to_csv('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/exchanges/max_m9.iJO1366.exchange_fluxes.tsv', sep='\t')
max_flux_samples.to_csv('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/exchanges/max.iJO1366.exchange_fluxes.tsv', sep='\t')
riptide_m9_media_flux_samples.to_csv('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/exchanges/riptide_m9.iJO1366.exchange_fluxes.tsv', sep='\t')
riptide_m9_flux_samples.to_csv('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/exchanges/riptide.iJO1366.exchange_fluxes.tsv', sep='\t')

### Comparing across conditions

In [12]:
# Aerobic growth in M9 + glucose
iJO1366_m9_aerobic = riptide.contextualize(iJO1366, transcriptome=m9_aerobic)


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 485 from 2583 (81.22% change)
Metabolites pruned to 481 from 1805 (73.35% change)
Flux through the objective DECREASED to ~85.9 from ~105.77 (18.79% change)
Context-specific metabolism correlates with transcriptome (p<<0.001 ***)

RIPTiDe completed in 1 minutes and 50 seconds



In [13]:
# Anaerobic growth in M9 + glucose
iJO1366.reactions.get_by_id('EX_o2_e').lower_bound = 0.0 # make anaerobic
iJO1366_m9_anaerobic = riptide.contextualize(iJO1366, transcriptome=m9_anaerobic)
iJO1366.reactions.get_by_id('EX_o2_e').lower_bound = -1000.0 # revert change


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 491 from 2583 (80.99% change)
Metabolites pruned to 491 from 1805 (72.8% change)
Flux through the objective DECREASED to ~86.8 from ~105.76 (17.93% change)
Context-specific metabolism correlates with transcriptome (p<<0.001 ***)

RIPTiDe completed in 2 minutes and 12 seconds



In [14]:
# Aerobic growth in LB
iJO1366_lb_aerobic = riptide.contextualize(iJO1366, transcriptome=lb_aerobic)


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 455 from 2583 (82.38% change)
Metabolites pruned to 456 from 1805 (74.74% change)
Flux through the objective DECREASED to ~86.58 from ~105.77 (18.14% change)
Context-specific metabolism correlates with transcriptome (p<<0.001 ***)

RIPTiDe completed in 2 minutes and 2 seconds



In [10]:
# Compare to base implementation of pFBA
# All coefficients set to 1.0
pfba_transcriptome = {}
for gene in lb_aerobic.keys(): pfba_transcriptome[gene] = 1.0
iJO1366_pfba = riptide.contextualize(iJO1366, transcriptome=pfba_transcriptome)


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 453 from 2583 (82.46% change)
Metabolites pruned to 453 from 1805 (74.9% change)
Flux through the objective DECREASED to ~86.89 from ~105.77 (17.85% change)

RIPTiDe completed in 1 minutes and 33 seconds



In [24]:
# Compares lists to create diagrams for 4 groups
def venn_comparison(list1, list2, list3, list4):
        
    # Confirm correct data types
    list1 = set(list1)
    list2 = set(list2)
    list3 = set(list3)
    list4 = set(list4)
    
    # Identify exclusive elements
    list1_only = list1.difference(list2)
    list1_only = list1_only.difference(list3)
    list1_only = list1_only.difference(list4)
    list2_only = list2.difference(list1)
    list2_only = list2_only.difference(list3)
    list2_only = list2_only.difference(list4)
    list3_only = list3.difference(list1)
    list3_only = list3_only.difference(list2)
    list3_only = list3_only.difference(list4)
    list4_only = list4.difference(list1)
    list4_only = list4_only.difference(list2)
    list4_only = list4_only.difference(list3)

    # Find overlap between just 2 groups
    list1_list2_overlap = list1.intersection(list2)
    list1_list2_overlap = list1_list2_overlap.difference(list3)
    list1_list2_overlap = list1_list2_overlap.difference(list4)
    list1_list3_overlap = list1.intersection(list3)
    list1_list3_overlap = list1_list3_overlap.difference(list2)
    list1_list3_overlap = list1_list3_overlap.difference(list4)
    list1_list4_overlap = list1.intersection(list4)
    list1_list4_overlap = list1_list4_overlap.difference(list2)
    list1_list4_overlap = list1_list4_overlap.difference(list3)
    list2_list3_overlap = list2.intersection(list3)
    list2_list3_overlap = list2_list3_overlap.difference(list1)
    list2_list3_overlap = list2_list3_overlap.difference(list4)
    list2_list4_overlap = list2.intersection(list4)
    list2_list4_overlap = list2_list4_overlap.difference(list1)
    list2_list4_overlap = list2_list4_overlap.difference(list3)
    list3_list4_overlap = list3.intersection(list4)
    list3_list4_overlap = list3_list4_overlap.difference(list1)
    list3_list4_overlap = list3_list4_overlap.difference(list2)

    # Find overlap in 3 groups
    list1_list2_list3_overlap = list1.intersection(list2)
    list1_list2_list3_overlap = list1_list2_list3_overlap.intersection(list3)
    list1_list2_list3_overlap = list1_list2_list3_overlap.difference(list4)
    list1_list2_list4_overlap = list1.intersection(list2)
    list1_list2_list4_overlap = list1_list2_list4_overlap.intersection(list4)
    list1_list2_list4_overlap = list1_list2_list4_overlap.difference(list3)
    list1_list3_list4_overlap = list1.intersection(list3)
    list1_list3_list4_overlap = list1_list3_list4_overlap.intersection(list4)
    list1_list3_list4_overlap = list1_list3_list4_overlap.difference(list2)
    list2_list3_list4_overlap = list2.intersection(list3)
    list2_list3_list4_overlap = list2_list3_list4_overlap.intersection(list4)
    list2_list3_list4_overlap = list2_list3_list4_overlap.difference(list1)
    
    # Find overlap between all groups
    all_list_overlap = list1.intersection(list2)
    all_list_overlap = all_list_overlap.intersection(list3)
    all_list_overlap = all_list_overlap.intersection(list4)
    
    # Calculate totals in each group
    list1_total = float(len(list1))
    list2_total = float(len(list2))
    list3_total = float(len(list3))
    list4_total = float(len(list4))
    list1_only_total = float(len(list1_only))
    list2_only_total = float(len(list2_only))
    list3_only_total = float(len(list3_only))
    list4_only_total = float(len(list4_only))
    list1_list2_overlap_total = float(len(list1_list2_overlap))
    list1_list3_overlap_total = float(len(list1_list3_overlap))
    list1_list4_overlap_total = float(len(list1_list4_overlap))
    list2_list3_overlap_total = float(len(list2_list3_overlap))
    list2_list4_overlap_total = float(len(list2_list4_overlap))
    list3_list4_overlap_total = float(len(list3_list4_overlap))
    list1_list2_list3_overlap_total = float(len(list1_list2_list3_overlap))
    list1_list2_list4_overlap_total = float(len(list1_list2_list4_overlap))
    list1_list3_list4_overlap_total = float(len(list1_list3_list4_overlap))
    list2_list3_list4_overlap_total = float(len(list2_list3_list4_overlap))
    all_list_overlap_total = float(len(all_list_overlap))
    
    # Calculate percent overlaps
    list1_only_percent = round(((list1_only_total / list1_total) * 100.0), 1)
    list2_only_percent = round(((list2_only_total / list2_total) * 100.0), 1)
    list3_only_percent = round(((list3_only_total / list3_total) * 100.0), 1)
    list4_only_percent = round(((list4_only_total / list4_total) * 100.0), 1)
    temp1 = (list1_list2_overlap_total / list1_total) * 100.0
    temp2 = (list1_list2_overlap_total / list2_total) * 100.0
    list1_list2_overlap_percent = round(numpy.mean([temp1, temp2]), 1)
    temp1 = (list1_list3_overlap_total / list1_total) * 100.0
    temp2 = (list1_list3_overlap_total / list3_total) * 100.0
    list1_list3_overlap_percent = round(numpy.mean([temp1, temp2]), 1)
    temp1 = (list1_list4_overlap_total / list1_total) * 100.0
    temp2 = (list1_list4_overlap_total / list4_total) * 100.0
    list1_list4_overlap_percent = round(numpy.mean([temp1, temp2]), 1)
    temp1 = (list2_list3_overlap_total / list2_total) * 100.0
    temp2 = (list2_list3_overlap_total / list3_total) * 100.0
    list2_list3_overlap_percent = round(numpy.mean([temp1, temp2]), 1)
    temp1 = (list2_list4_overlap_total / list2_total) * 100.0
    temp2 = (list2_list4_overlap_total / list4_total) * 100.0
    list2_list4_overlap_percent = round(numpy.mean([temp1, temp2]), 1)
    temp1 = (list3_list4_overlap_total / list3_total) * 100.0
    temp2 = (list3_list4_overlap_total / list4_total) * 100.0
    list3_list4_overlap_percent = round(numpy.mean([temp1, temp2]), 1)
    temp1 = (list1_list2_list3_overlap_total / list1_total) * 100.0
    temp2 = (list1_list2_list3_overlap_total / list2_total) * 100.0
    temp3 = (list1_list2_list3_overlap_total / list3_total) * 100.0
    list1_list2_list3_overlap_percent = round(numpy.mean([temp1, temp2, temp3]), 1)
    temp1 = (list1_list2_list4_overlap_total / list1_total) * 100.0
    temp2 = (list1_list2_list4_overlap_total / list2_total) * 100.0
    temp3 = (list1_list2_list4_overlap_total / list4_total) * 100.0
    list1_list2_list4_overlap_percent = round(numpy.mean([temp1, temp2, temp3]), 1)
    temp1 = (list1_list3_list4_overlap_total / list1_total) * 100.0
    temp2 = (list1_list3_list4_overlap_total / list3_total) * 100.0
    temp3 = (list1_list3_list4_overlap_total / list4_total) * 100.0
    list1_list3_list4_overlap_percent = round(numpy.mean([temp1, temp2, temp3]), 1)
    temp1 = (list2_list3_list4_overlap_total / list2_total) * 100.0
    temp2 = (list2_list3_list4_overlap_total / list3_total) * 100.0
    temp3 = (list2_list3_list4_overlap_total / list4_total) * 100.0
    list2_list3_list4_overlap_percent = round(numpy.mean([temp1, temp2, temp3]), 1)
    temp1 = (all_list_overlap_total / list1_total) * 100.0
    temp2 = (all_list_overlap_total / list2_total) * 100.0
    temp3 = (all_list_overlap_total / list3_total) * 100.0
    temp4 = (all_list_overlap_total / list4_total) * 100.0
    all_list_overlap_percent = round(numpy.mean([temp1, temp2, temp3, temp4]), 1)
    
    # Print report to the screen
    print('List 1 only: ' + str(list1_only_percent) + '% (' + str(int(list1_only_total)) + ')')
    print('List 2 only: ' + str(list2_only_percent) + '% (' + str(int(list2_only_total)) + ')')
    print('List 3 only: ' + str(list3_only_percent) + '% (' + str(int(list3_only_total)) + ')')
    print('List 4 only: ' + str(list4_only_percent) + '% (' + str(int(list4_only_total)) + ')')
    print('')
    print('List 1 + List 2: ' + str(list1_list2_overlap_percent) + '% (' + str(int(list1_list2_overlap_total)) + ')')
    print('List 1 + List 3: ' + str(list1_list3_overlap_percent) + '% (' + str(int(list1_list3_overlap_total)) + ')')
    print('List 1 + List 4: ' + str(list1_list4_overlap_percent) + '% (' + str(int(list1_list4_overlap_total)) + ')')
    print('List 2 + List 3: ' + str(list2_list3_overlap_percent) + '% (' + str(int(list2_list3_overlap_total)) + ')')
    print('List 2 + List 4: ' + str(list2_list4_overlap_percent) + '% (' + str(int(list2_list4_overlap_total)) + ')')
    print('List 3 + List 4: ' + str(list3_list4_overlap_percent) + '% (' + str(int(list3_list4_overlap_total)) + ')')
    print('')
    print('List 1 + List 2 + List 3: ' + str(list1_list2_list3_overlap_percent) + '% (' + str(int(list1_list2_list3_overlap_total)) + ')')
    print('List 1 + List 2 + List 4: ' + str(list1_list2_list4_overlap_percent) + '% (' + str(int(list1_list2_list4_overlap_total)) + ')')
    print('List 1 + List 3 + List 4: ' + str(list1_list3_list4_overlap_percent) + '% (' + str(int(list1_list3_list4_overlap_total)) + ')')
    print('List 2 + List 3 + List 4: ' + str(list2_list3_list4_overlap_percent) + '% (' + str(int(list2_list3_list4_overlap_total)) + ')')
    print('')
    print('Shared: ' + str(all_list_overlap_percent) + '% (' + str(int(all_list_overlap_total)) + ')')

    # Return new lists
    return [list1_only,list2_only,list3_only,list4_only,list1_list2_overlap, list1_list3_overlap, list1_list4_overlap, list2_list3_overlap, list2_list4_overlap, list3_list4_overlap, list1_list2_list3_overlap, list1_list2_list4_overlap, list1_list3_list4_overlap, list2_list3_list4_overlap, all_list_overlap]


In [33]:
# Reactions
iJO1366_m9_aerobic_reactions = [x.id for x in iJO1366_m9_aerobic.model.reactions]
iJO1366_m9_anaerobic_reactions = [x.id for x in iJO1366_m9_anaerobic.model.reactions]
iJO1366_lb_aerobic_reactions = [x.id for x in iJO1366_lb_aerobic.model.reactions]
iJO1366_pfba_reactions = [x.id for x in iJO1366_pfba.model.reactions]

reactions_comparisons = venn_comparison(iJO1366_pfba_reactions, iJO1366_lb_aerobic_reactions, iJO1366_m9_aerobic_reactions, iJO1366_m9_anaerobic_reactions)

# List 1: pfba
# List 2: lb_aerobic
# List 3: m9_aerobic
# List 4: m9_anaerobic

List 1 only: 3.8% (17)
List 2 only: 4.3% (21)
List 3 only: 4.1% (19)
List 4 only: 8.0% (40)

List 1 + List 2: 4.5% (21)
List 1 + List 3: 4.8% (22)
List 1 + List 4: 1.3% (6)
List 2 + List 3: 1.7% (8)
List 2 + List 4: 8.8% (44)
List 3 + List 4: 5.8% (28)

List 1 + List 2 + List 3: 4.7% (22)
List 1 + List 2 + List 4: 3.1% (15)
List 1 + List 3 + List 4: 1.1% (5)
List 2 + List 3 + List 4: 4.5% (22)

Shared: 71.5% (341)


In [34]:
# Metabolites
iJO1366_m9_aerobic_metabolites = [x.id for x in iJO1366_m9_aerobic.model.metabolites]
iJO1366_m9_anaerobic_metabolites = [x.id for x in iJO1366_m9_anaerobic.model.metabolites]
iJO1366_lb_aerobic_metabolites = [x.id for x in iJO1366_lb_aerobic.model.metabolites]
iJO1366_pfba_metabolites = [x.id for x in iJO1366_pfba.model.metabolites]

metabolites_comparisons = venn_comparison(iJO1366_pfba_metabolites, iJO1366_lb_aerobic_metabolites, iJO1366_m9_aerobic_metabolites, iJO1366_m9_anaerobic_metabolites)

# List 1: pfba
# List 2: lb_aerobic
# List 3: m9_aerobic
# List 4: m9_anaerobic

List 1 only: 1.3% (6)
List 2 only: 3.2% (16)
List 3 only: 1.9% (9)
List 4 only: 5.0% (25)

List 1 + List 2: 2.1% (10)
List 1 + List 3: 2.6% (12)
List 1 + List 4: 0.6% (3)
List 2 + List 3: 1.0% (5)
List 2 + List 4: 6.8% (34)
List 3 + List 4: 3.5% (17)

List 1 + List 2 + List 3: 1.7% (8)
List 1 + List 2 + List 4: 1.9% (9)
List 1 + List 3 + List 4: 0.4% (2)
List 2 + List 3 + List 4: 2.9% (14)

Shared: 83.6% (399)


In [35]:
# Screen context specific growth rates by each model's optimal growth

# Determine bounds
pfba_obj_val_lb = iJO1366_pfba.model.slim_optimize() * 0.8
lb_aerobic_obj_val_lb = iJO1366_lb_aerobic.model.slim_optimize() * 0.8
m9_aerobic_obj_val_lb = iJO1366_m9_aerobic.model.slim_optimize() * 0.8
m9_anaerobic_obj_val_lb = iJO1366_m9_anaerobic.model.slim_optimize() * 0.8
pfba_obj_val_ub = iJO1366_pfba.model.slim_optimize()
lb_aerobic_obj_val_ub = iJO1366_lb_aerobic.model.slim_optimize()
m9_aerobic_obj_val_ub = iJO1366_m9_aerobic.model.slim_optimize()
m9_anaerobic_obj_val_ub = iJO1366_m9_anaerobic.model.slim_optimize()

# Collect fluxes
pfba_biomass = list(iJO1366_pfba.flux_samples['BIOMASS_Ec_iJO1366_WT_53p95M'])
lb_biomass = list(iJO1366_lb_aerobic.flux_samples['BIOMASS_Ec_iJO1366_WT_53p95M'])
m9_aerobic_biomass = list(iJO1366_m9_aerobic.flux_samples['BIOMASS_Ec_iJO1366_WT_53p95M'])
m9_anaerobic_biomass = list(iJO1366_m9_anaerobic.flux_samples['BIOMASS_Ec_iJO1366_WT_53p95M'])

# Screen fluxes
pfba_biomass = [x for x in pfba_biomass if x >= pfba_obj_val_lb and x <= pfba_obj_val_ub]
lb_biomass = [x for x in lb_biomass if x >= lb_aerobic_obj_val_lb and x <= lb_aerobic_obj_val_ub]
m9_aerobic_biomass = [x for x in m9_aerobic_biomass if x >= m9_aerobic_obj_val_lb and x <= m9_aerobic_obj_val_ub]
m9_anaerobic_biomass = [x for x in m9_anaerobic_biomass if x >= m9_anaerobic_obj_val_lb and x <= m9_anaerobic_obj_val_ub]

# Convert to per hour rate
pfba_rates = [round((x / 60.0), 3) for x in pfba_biomass]
lb_aerobic_rates = [round((x / 60.0), 3) for x in lb_biomass]
m9_aerobic_rates = [round((x / 60.0), 3) for x in m9_aerobic_biomass]
m9_anaerobic_rates = [round((x / 60.0), 3) for x in m9_anaerobic_biomass]

# Subsample evenly
import random
sub_level = min([len(pfba_rates), len(lb_aerobic_rates), len(m9_aerobic_rates), len(m9_anaerobic_rates)])
pfba_sub = random.sample(range(0,len(pfba_rates)), sub_level)
lb_sub = random.sample(range(0,len(lb_aerobic_rates)), sub_level)
m9a_sub = random.sample(range(0,len(m9_aerobic_rates)), sub_level)
m9n_sub = random.sample(range(0,len(m9_anaerobic_rates)), sub_level)
pfba_rates = [pfba_rates[i] for i in pfba_sub]
lb_aerobic_rates = [lb_aerobic_rates[i] for i in lb_sub]
m9_aerobic_rates = [m9_aerobic_rates[i] for i in m9a_sub]
m9_anaerobic_rates = [m9_anaerobic_rates[i] for i in m9n_sub]

# Convert to strings
pfba_rates = [str(x) for x in pfba_rates]
pfba_rates = 'base_pfba\t' + '\t'.join(pfba_rates) + '\n'
m9_aerobic_rates = [str(x) for x in m9_aerobic_rates]
m9_aerobic_rates = 'm9_gluc_aerobic\t' + '\t'.join(m9_aerobic_rates) + '\n'
m9_anaerobic_rates = [str(x) for x in m9_anaerobic_rates]
m9_anaerobic_rates = 'm9_gluc_anaerobic\t' + '\t'.join(m9_anaerobic_rates) + '\n'
lb_aerobic_rates = [str(x) for x in lb_aerobic_rates]
lb_aerobic_rates = 'lb_aerobic\t' + '\t'.join(lb_aerobic_rates) + '\n'

# Write to file
with open('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/new_growth_rates.tsv', 'w') as rates:
    rates.write(pfba_rates)
    rates.write(m9_aerobic_rates)
    rates.write(m9_anaerobic_rates)
    rates.write(lb_aerobic_rates)

In [19]:
# Write contextualized models to SBMLs and JSONs
cobra.io.write_sbml_model(iJO1366_m9_aerobic, '/home/matt/Desktop/repos/Jenior_RIPTiDe_2019/data/riptide_models/iJO1366_m9_aerobic.sbml')
cobra.io.save_json_model(iJO1366_m9_aerobic, '/home/matt/Desktop/repos/Jenior_RIPTiDe_2019/data/riptide_models/iJO1366_m9_aerobic.json')
cobra.io.write_sbml_model(iJO1366_m9_anaerobic, '/home/matt/Desktop/repos/Jenior_RIPTiDe_2019/data/riptide_models/iJO1366_m9_anaerobic.sbml')
cobra.io.save_json_model(iJO1366_m9_anaerobic, '/home/matt/Desktop/repos/Jenior_RIPTiDe_2019/data/riptide_models/iJO1366_m9_anaerobic.json')
cobra.io.write_sbml_model(iJO1366_lb_aerobic, '/home/matt/Desktop/repos/Jenior_RIPTiDe_2019/data/riptide_models/iJO1366_lb_aerobic.sbml')
cobra.io.save_json_model(iJO1366_lb_aerobic, '/home/matt/Desktop/repos/Jenior_RIPTiDe_2019/data/riptide_models/iJO1366_lb_aerobic.json')

In [21]:
# Correct the sample labels
def label_flux_samples(file_name, label):
    new_name = file_name.rstrip('tsv') + 'format.tsv'
    new_file = open(new_name, 'w')
    
    with open(file_name, 'r') as samples:
        header = samples.readline()
        header = 'sample\t' + header
        new_file.write(header)
        current = 1
        
        for line in samples:
            line = label + '_' + str(current) + '\t' + line
            new_file.write(line)
            current += 1

    new_file.close()

In [27]:
# Write chosen flux sample tables to tsvs
m9_aerobic_samples.to_csv('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/M9_aerobic.flux_samples.tsv', sep='\t')
m9_anaerobic_samples.to_csv('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/M9_anaerobic.flux_samples.tsv', sep='\t')
lb_samples.to_csv('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/LB_aerobic.flux_samples.tsv', sep='\t')
pfba_samples.to_csv('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/pFBA.flux_samples.tsv', sep='\t')

    
label_flux_samples('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/M9_aerobic.flux_samples.tsv', 'm9_aer')
label_flux_samples('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/M9_anaerobic.flux_samples.tsv', 'm9_anaer')
label_flux_samples('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/LB_aerobic.flux_samples.tsv', 'lb')
label_flux_samples('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/pFBA.flux_samples.tsv', 'pfba')

### Context-specific Essentiality

In [8]:
import cobra.flux_analysis

In [19]:
iJO1366_essential_genes = cobra.flux_analysis.find_essential_genes(iJO1366)
print('Essential genes: ' + str(len(iJO1366_essential_genes)))

Essential genes: 105


In [20]:
iJO1366_pfba_essential_genes = cobra.flux_analysis.find_essential_genes(iJO1366_pfba)
iJO1366_pfba_essential_genes = set([x.id for x in iJO1366_pfba_essential_genes])
print('Essential genes: ' + str(len(iJO1366_pfba_essential_genes)))

Essential genes: 226


In [21]:
iJO1366_lb_aerobic_essential_genes = cobra.flux_analysis.find_essential_genes(iJO1366_lb_aerobic)
iJO1366_lb_aerobic_essential_genes = set([x.id for x in iJO1366_lb_aerobic_essential_genes])
print('Essential genes: ' + str(len(iJO1366_lb_aerobic_essential_genes)))

Essential genes: 227


In [22]:
iJO1366_m9_aerobic_essential_genes = cobra.flux_analysis.find_essential_genes(iJO1366_m9_aerobic)
iJO1366_m9_aerobic_essential_genes = set([x.id for x in iJO1366_m9_aerobic_essential_genes])
print('Essential genes: ' + str(len(iJO1366_m9_aerobic_essential_genes)))

Essential genes: 232


In [23]:
iJO1366_m9_anaerobic_essential_genes = cobra.flux_analysis.find_essential_genes(iJO1366_m9_anaerobic)
iJO1366_m9_anaerobic_essential_genes = set([x.id for x in iJO1366_m9_anaerobic_essential_genes])
print('Essential genes: ' + str(len(iJO1366_m9_anaerobic_essential_genes)))

Essential genes: 229


In [14]:
essentiality_comparisons = venn_comparison(iJO1366_pfba_essential_genes, iJO1366_lb_aerobic_essential_genes, iJO1366_m9_aerobic_essential_genes, iJO1366_m9_anaerobic_essential_genes)
# List 1: pfba
# List 2: lb_aerobic
# List 3: m9_aerobic
# List 4: m9_anaerobic

NameError: name 'venn_comparison' is not defined

In [15]:
# Find those genes shared in all models
core_essential = iJO1366_pfba_essential_genes.intersection(iJO1366_lb_aerobic_essential_genes)
core_essential = core_essential.intersection(iJO1366_m9_aerobic_essential_genes)
core_essential = core_essential.intersection(iJO1366_m9_anaerobic_essential_genes)
print('Essential in all GENREs: ' + str(len(core_essential)))

# Substract as background from each
iJO1366_pfba_essential_genes = iJO1366_pfba_essential_genes.difference(core_essential)
iJO1366_lb_aerobic_essential_genes = iJO1366_lb_aerobic_essential_genes.difference(core_essential)
iJO1366_m9_aerobic_essential_genes = iJO1366_m9_aerobic_essential_genes.difference(core_essential)
iJO1366_m9_anaerobic_essential_genes = iJO1366_m9_anaerobic_essential_genes.difference(core_essential)

# Find non-essentiality across models
iJO1366_pfba_genes = set([x.id for x in iJO1366_pfba.genes])
iJO1366_lb_aerobic_genes = set([x.id for x in iJO1366_lb_aerobic.genes])
iJO1366_m9_aerobic_genes = set([x.id for x in iJO1366_m9_aerobic.genes])
iJO1366_m9_anaerobic_genes = set([x.id for x in iJO1366_m9_anaerobic.genes])

# Compare overlapping genes
total_genes = set()
total_genes |= iJO1366_pfba_essential_genes
total_genes |= iJO1366_lb_aerobic_essential_genes
total_genes |= iJO1366_m9_aerobic_essential_genes
total_genes |= iJO1366_m9_anaerobic_essential_genes
with open('/home/matt/Desktop/repos/Jenior_RIPTiDe_2019/data/essentiality_test.tsv', 'w') as outfile:
    outfile.write('gene\tpfba\tlb_aerobic\tm9_aerobic\tm9_anaerobic\n')
    for gene in total_genes:
        entry = ['filler','filler','filler','filler']
        
        if gene in iJO1366_pfba_essential_genes:
            entry[0] = 2
        elif gene in iJO1366_pfba_genes:
            entry[0] = 1
        else:
            entry[0] = 0
            
        if gene in iJO1366_lb_aerobic_essential_genes:
            entry[1] = 2
        elif gene in iJO1366_lb_aerobic_genes:
            entry[1] = 1
        else:
            entry[1] = 0
            
        if gene in iJO1366_m9_aerobic_essential_genes:
            entry[2] = 2
        elif gene in iJO1366_m9_aerobic_genes:
            entry[2] = 1
        else:
            entry[2] = 0
            
        if gene in iJO1366_m9_anaerobic_essential_genes:
            entry[3] = 2
        elif gene in iJO1366_m9_anaerobic_genes:
            entry[3] = 1
        else:
            entry[3] = 0
            
        entry = gene + '\t' + '\t'.join([str(x) for x in entry]) + '\n'
        outfile.write(entry)


Essential in all GENREs: 178


### Metatranscriptomic analysis

In [69]:
def find_source(model, met_id):
    generating = set()
    for rxn in model.reactions:
        for met in rxn.products:
            if met_id in met.id:
                generating |= set([rxn.id])
        
    print('Metabolite sources: ' + str(len(generating)))
    return generating

In [5]:
clinda_k12_metaT = {}
with open('/home/matt/Desktop/repos/Jenior_RIPTiDe_2019/data/transcript/clinda_k12.mapped.norm.tsv', 'r') as transcription:
    header = transcription.readline()
    for line in transcription:
        line = line.split()
        clinda_k12_metaT[line[0]] = float(line[2])

In [6]:
iJO1366_invivo = riptide.contextualize(iJO1366, transcriptome=clinda_k12_metaT)


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...
[0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.813389956740874, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447, 0.7153049700423447,

In [29]:
invivo_rates = collect_growth_rates(invivo_metaT_samples, 'BIOMASS_Ec_iJO1366_WT_53p95M')
print([min(invivo_rates), numpy.median(invivo_rates), max(invivo_rates)])
with open('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/invivo_growth_rates.tsv', 'w') as output_file:
    for x in invivo_rates: output_file.write(str(x) + '\n')

[0.538, 0.578, 0.886]


In [36]:
invivo_anaerobic_atp = find_source(iJO1366_invivo_metaT, 'atp_c')

Metabolite sources: 3


In [54]:
vitro_ex = set([x.id for x in iJO1366_m9_anaerobic.reactions if 'EX_' in x.id])
vivo_ex = set([x.id for x in iJO1366_invivo_metaT.reactions if 'EX_' in x.id])

vitro_ex_only = vitro_ex.difference(vivo_ex)
vitro_ex_only_input = set()
for y in vitro_ex_only:
    if abs(iJO1366_m9_anaerobic.reactions.get_by_id(y).lower_bound) > abs(iJO1366_m9_anaerobic.reactions.get_by_id(y).upper_bound):
        vitro_ex_only_input |= set([y])

vivo_ex_only = vivo_ex.difference(vitro_ex)
vivo_ex_only_input = set()
for y in vivo_ex_only:
    if abs(iJO1366_invivo_metaT.reactions.get_by_id(y).lower_bound) > abs(iJO1366_invivo_metaT.reactions.get_by_id(y).upper_bound):
        vivo_ex_only_input |= set([y])

In [58]:
for x in vitro_ex_only_input: print(iJO1366_m9_anaerobic.reactions.get_by_id(x).reactants[0].name)

Quinate
Glycerol 3-phosphate
Glycerophosphoserine
L-Leucine
L-tartrate
Ethanol
Guanosine
D-Fructose 6-phosphate
Allantoin


In [59]:
for x in vivo_ex_only_input: print(iJO1366_invivo_metaT.reactions.get_by_id(x).reactants[0].name)

L-Serine
Deoxyuridine
Trehalose
Deoxyguanosine
Glycine
Pyridoxine
Myo-Inositol hexakisphosphate
Shikimate


In [19]:
max_doubling_time(iJO1366_invivo_metaT)

1.11 hours


In [22]:
# Write flux sample tables to tsv
m9_anaerobic_samples.to_csv('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/invitro.flux_samples.tsv', sep='\t')
label_flux_samples('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/invitro.flux_samples.tsv', 'invitro')

invivo_metaT_samples.to_csv('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/invivo.flux_samples.tsv', sep='\t')
label_flux_samples('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/invivo.flux_samples.tsv', 'invivo')

In [17]:
invivo_metaT_times = collect_doubling_times(invivo_metaT_samples, 'BIOMASS_Ec_iJO1366_WT_53p95M')
print([min(invivo_metaT_times), numpy.median(invivo_metaT_times), max(invivo_metaT_times)])
invivo_metaT_rates = collect_growth_rates(invivo_metaT_samples, 'BIOMASS_Ec_iJO1366_WT_53p95M')
print([min(invivo_metaT_rates), numpy.median(invivo_metaT_rates), max(invivo_metaT_rates)])

[86.41, 105.13, 111.6]
[0.538, 0.571, 0.694]


In [70]:
m9 = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/riptide_models/iJO1366_m9_aerobic.sbml')
lb = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/riptide_models/iJO1366_lb_aerobic.sbml')

In [71]:
m9_nadph = find_source(m9, 'nadph_c')
lb_nadph = find_source(lb, 'nadph_c')

Metabolite sources: 4
Metabolite sources: 3


### Testing Previous Integration Algorithms

In [6]:
iJO1366 = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/reconstructions/iJO1366.xml')
iJO1366.objective = iJO1366.reactions.BIOMASS_Ec_iJO1366_WT_53p95M

# Open all exchanges
exchanges = set()
for rxn in iJO1366.reactions:
    if len(rxn.reactants) == 0 or len(rxn.products) == 0:
        rxn.bounds = (min(rxn.lower_bound, -1000), max(rxn.upper_bound, 1000))
        exchanges |= set([rxn.id])

In [7]:
# Comparison to GIMME, iMAT, and CORDA
import copy
import cobra
from driven.flux_analysis.transcriptomics import *
from driven.data_sets.expression_profile import *
from corda import CORDA

In [8]:
# Read in formatted data
m9_aerobic_driven = ExpressionProfile.from_csv('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/transcript/Monk_et_al_2016/m9_aerobic_expression.csv')

# Define thresholds
gimme_threshold = 12

imat_lo_threshold = 10
imat_hi_threshold = 900

conda_lo_threshold = 10
conda_hi_threshold = 900

# Format data for CORDA
conda_gene_confidences = {}
with open('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/transcript/Monk_et_al_2016/m9_aerobic_expression.csv', 'r') as transcription:
    for line in transcription:
        gene = line.strip().split(',')[0]
        abundance = float(line.strip().split(',')[1])
        
        if abundance < conda_lo_threshold:
            confidence = 1
        elif abundance >= conda_lo_threshold and abundance < conda_hi_threshold:
            confidence = 2
        elif abundance >= conda_hi_threshold:
            confidence = 3
        else:
            confidence = 0
        
        conda_gene_confidences[gene] = confidence

conda_rxn_confidences = {}
conda_rxn_confidences['BIOMASS_Ec_iJO1366_WT_53p95M'] = 3
for gene in iJO1366.genes:
    try:
        confidence = conda_gene_confidences[gene.id]
    except:
        continue
    reactions = [x.id for x in list(gene.reactions)]
    for rxn in reactions:
        conda_rxn_confidences[rxn] = confidence
        
for rxn in iJO1366.reactions:
    if rxn.id not in conda_rxn_confidences.keys():
        conda_rxn_confidences[rxn.id] = 0


In [12]:
# GIMME
iJO1366_gimme_result = gimme(iJO1366, cutoff=gimme_threshold, expression_profile=m9_aerobic_driven)
print('Done')

AssertionError: 

In [33]:
# Check for overlap with lowly expressed reactions and gapfilled
current = 0
remove = []
for entry in iJO1366_gimme_result.data_frame['inconsistency_scores']:
    if entry != 0.0:
        remove.append(list(iJO1366_gimme_result.fluxes.index)[current])
    current += 1

iJO1366_gimme = copy.deepcopy(iJO1366)

# Constrain fluxes to match output
for rxn_id, flux in iJO1366_gimme_result.fluxes.items():
    iJO1366_gimme.reactions.get_by_id(rxn_id).bounds = (flux, flux)
    
# Prune inactive reactions
for rxn in remove:
    iJO1366_gimme.reactions.get_by_id(rxn).remove_from_model(remove_orphans=True)
iJO1366_gimme.reactions.get_by_id('DM_4crsol_c').remove_from_model(remove_orphans=True)
print(len(remove))
removed = 1
while removed == 1:
    removed = 0
    for cpd in iJO1366_gimme.metabolites:
        if len(cpd.reactions) == 0:
            cpd.remove_from_model(); removed = 1
    for rxn in iJO1366_gimme.reactions:
        if len(rxn.metabolites) == 0: 
            rxn.remove_from_model(); removed = 1

# Test for growth
print('Objective value: ' + str(iJO1366_gimme.slim_optimize()))

130
Objective value: nan


In [13]:
# iMAT
iJO1366_imat_result = imat(iJO1366, expression_profile=m9_aerobic_driven, low_cutoff=imat_lo_threshold, high_cutoff=imat_hi_threshold)
print('Done')

AssertionError: 

In [35]:
# Check for overlap with lowly expressed reactions
from cobra.flux_analysis.parsimonious import *

imat_pfba = pfba(iJO1366, fraction_of_optimum=0.8)
test1 = imat_pfba.fluxes[imat_pfba.fluxes > 0.0]
test1 = set(test1.index)

test2 = set()
for index in iJO1366_imat_result.lowly_express.keys():
    if iJO1366_imat_result.lowly_express[index] == True:
        test2 |= set([index])

gapfilled_imat = test1.intersection(test2)
print(len(gapfilled_imat))

97


In [9]:
# CORDA
optimal = CORDA(iJO1366, conda_rxn_confidences)
optimal.build()
print('Done')

Done


In [10]:
print(optimal)

build status: reconstruction complete
Inc. reactions: 1001/2583
 - unclear: 231/621
 - exclude: 0/0
 - low and medium: 611/1784
 - high: 159/178



In [11]:
optimal.model.objective = optimal.model.reactions.BIOMASS_Ec_iJO1366_WT_53p95M
optimal.model.slim_optimize()

0.0